# Model Training 

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
#from catboost import CatBoostRegressor
#from xgboost import XGBRegressor
import warnings

In [2]:
df = pd.read_csv('StudentsPerformance.csv')
df.head(5)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## Preparing X and y variables

In [3]:
X = df.drop(columns=['math score', 'reading score', 'writing score'], axis=1)
X.head(5)

,gender,race/ethnicity,parental level of education,lunch,test preparation course
0,female,group B,bachelor's degree,standard,none
1,female,group C,some college,standard,completed
2,female,group B,master's degree,standard,none
3,male,group A,associate's degree,free/reduced,none
4,male,group C,some college,standard,none


In [4]:
y = df[['math score', 'reading score', 'writing score']]
y.head(5)

,math score,reading score,writing score
0,72,72,74
1,69,90,88
2,90,95,93
3,47,57,44
4,76,78,75


## Feature Engineering and Data Preprocessing

In [5]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['gender'].unique())

print("Categories in 'race/ethnicity' variable:  ",end=" ")
print(df['race/ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental level of education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test preparation course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race/ethnicity' variable:   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable:      ['standard' 'free/reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", categorical_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

X = preprocessor.fit_transform(X)

In [7]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5000 stored elements and shape (1000, 17)>

In [8]:
X.shape

(1000, 17)

### separate dataset into train and test

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 17), (200, 17))

# Linear Regression

In [44]:
# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions 
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Evaluate model performance 
def evaluate_model(true, predicted): 
    mae = mean_absolute_error(true, predicted, multioutput='raw_values') # multioutput: By including all three scores in the target variable (y), the linear regression model is performing multi-output regression.
    mse = mean_squared_error(true, predicted, multioutput='raw_values') 
    rmse = np.sqrt(mse) 
    r2_square = r2_score(true, predicted, multioutput='raw_values')
    return mae, rmse, r2_square 
    
# Get performance metrics
train_mae, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)  
test_mae, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred) 

# Print performance metrics for each score 
print('Model performance for Training set')
print(f'MAE (Writing, Reading, Math): {train_mae}') 
print(f'RMSE (Writing, Reading, Math): {train_rmse}') 
print(f'R^2 (Writing, Reading, Math): {train_r2}')

print('----------------------------------')
    
print('Model performance for Test set')
print(f'MAE (Writing, Reading, Math): {test_mae}') 
print(f'RMSE (Writing, Reading, Math): {test_rmse}') 
print(f'R^2 (Writing, Reading, Math): {test_r2}')

lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = r2_score(y_test, y_test_pred)*100
print(" Accuracy of the model is %.2f" %score)



Model performance for Training set
MAE (Writing, Reading, Math): [10.46971818 10.23436341  9.9721934 ]
RMSE (Writing, Reading, Math): [12.85854981 12.62747865 12.20173389]
R^2 (Writing, Reading, Math): [0.26660392 0.23660746 0.34494769]
----------------------------------
Model performance for Test set
MAE (Writing, Reading, Math): [11.26987358 10.83028806 10.19314286]
RMSE (Writing, Reading, Math): [14.16018658 13.79186427 13.32111664]
R^2 (Writing, Reading, Math): [0.17599983 0.15939614 0.26373589]
 Accuracy of the model is 19.97


# Ridge Regression 

In [37]:
#initialize mmodel
model = Ridge()
#fit model on train set
model.fit(X_train, y_train)
#Predict 
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
#evaluate model
train_mae, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)  
test_mae, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred) 

# Print performance metrics for each score 
print('Model performance for Training set')
print(f'MAE (Writing, Reading, Math): {train_mae}') 
print(f'RMSE (Writing, Reading, Math): {train_rmse}') 
print(f'R^2 (Writing, Reading, Math): {train_r2}')

print('----------------------------------')
    
print('Model performance for Test set')
print(f'MAE (Writing, Reading, Math): {test_mae}') 
print(f'RMSE (Writing, Reading, Math): {test_rmse}') 
print(f'R^2 (Writing, Reading, Math): {test_r2}')


Model performance for Training set
MAE (Writing, Reading, Math): [10.46912652 10.23389821  9.9723167 ]
RMSE (Writing, Reading, Math): [12.85859846 12.62751319 12.20179338]
R^2 (Writing, Reading, Math): [0.26659837 0.23660328 0.3449413 ]
----------------------------------
Model performance for Test set
MAE (Writing, Reading, Math): [11.27121391 10.83350195 10.19486472]
RMSE (Writing, Reading, Math): [14.16045995 13.79361411 13.32437073]
R^2 (Writing, Reading, Math): [0.17596801 0.15918282 0.26337614]


# K-Neighbors Regressor